# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups: 

1. Retrieving Feature Groups
2. Feature Group investigation
3. Transformation functions
4. Feature Views
5. Training Datasets
6. Training Datasets with Event Time filter



![tutorial-flow](images/02_training-dataset.png) 

---

## <span style="color:#ff5f27;"> 🔮 🪝 Connecting to Feature Store and Retrieving Feature Groups </span>

In [1]:
import hsfs

conn = hsfs.connection()

fs = conn.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


> In order to retrieve necessary Feature Group we can use `FeatureStore.get_or_create_feature_group()` method.

In [2]:
fg_weather = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

In [3]:
fg_calendar = fs.get_or_create_feature_group(
    name = 'calendar_fg',
    version = 1
)

In [4]:
fg_electricity = fs.get_or_create_feature_group(
    name = 'electricity_fg',
    version = 1
)

---

# <span style="color:#ff5f27;">🕵🏻‍♂️ Feature Groups Investigation</span>

We can use `FeatureGroup.show()` method to select top n rows. 

Also we use method `FeatureGroup.read()` in order **to aggregate queries**, which are the output of next methods:

- `FeatureGroup.get_feature()` to get specific feature from our Feature Group.

- `FeatureGroup.select()` to get a subset of features from our Feature Group.

- `FeatureGroup.select_all()` to get all features from our Feature Group.

- `FeatureGroup.select_except()` to get all features except a few from our Feature Group.

- `FeatureGroup.filter()` to apply specific filter to the feature group.

In [5]:
fg_weather.select_all()

In [6]:
fg_weather.select_all().read().head()

2022-07-26 08:46:03,633 INFO: USE `electricity_demand_featurestore`
2022-07-26 08:46:04,353 INFO: SELECT `fg0`.`index` `index`, `fg0`.`date` `date`, `fg0`.`min_temperature` `min_temperature`, `fg0`.`max_temperature` `max_temperature`, `fg0`.`solar_exposure` `solar_exposure`, `fg0`.`rainfall` `rainfall`, `fg0`.`day_of_week` `day_of_week`, `fg0`.`day_of_month` `day_of_month`, `fg0`.`day_of_year` `day_of_year`, `fg0`.`week_of_year` `week_of_year`, `fg0`.`month` `month`, `fg0`.`quarter` `quarter`, `fg0`.`year` `year`
FROM `electricity_demand_featurestore`.`weather_fg_1` `fg0`


,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,month,quarter,year
0,1886,1583020800000,13.2,32.1,20.8,0.0,6,1,61,9,3,1,2020
1,2039,1596240000000,5.7,16.4,11.1,0.0,5,1,214,31,8,3,2020
2,1347,1536451200000,8.6,18.1,7.3,0.0,6,9,252,36,9,3,2018
3,634,1474848000000,8.1,16.9,13.6,0.6,0,26,270,39,9,3,2016
4,8,1420761600000,16.5,18.0,3.1,1.2,4,9,9,2,1,1,2015


In [7]:
fg_calendar.select_except(['index']).show(5)

2022-07-26 08:46:06,001 INFO: USE `electricity_demand_featurestore`
2022-07-26 08:46:06,734 INFO: SELECT `fg0`.`date` `date`, `fg0`.`school_day` `school_day`, `fg0`.`holiday` `holiday`
FROM `electricity_demand_featurestore`.`calendar_fg_1` `fg0`


,date,school_day,holiday
0,1583020800000,1,0
1,1596240000000,1,0
2,1536451200000,1,0
3,1474848000000,0,0
4,1420761600000,0,0


In [8]:
fg_electricity.select('demand').show(5)

2022-07-26 08:46:08,333 INFO: USE `electricity_demand_featurestore`
2022-07-26 08:46:09,096 INFO: SELECT `fg0`.`demand` `demand`
FROM `electricity_demand_featurestore`.`electricity_fg_1` `fg0`


,demand
0,101413.145
1,111590.650
2,107838.950
3,124343.190
4,135452.260


In [9]:
fg_electricity.filter(fg_electricity.demand > 10000).show(5)

2022-07-26 08:46:10,787 INFO: USE `electricity_demand_featurestore`
2022-07-26 08:46:11,550 INFO: SELECT `fg0`.`index` `index`, `fg0`.`date` `date`, `fg0`.`rrp` `rrp`, `fg0`.`frac_at_neg_rrp` `frac_at_neg_rrp`, `fg0`.`demand` `demand`, `fg0`.`rrp_positive` `rrp_positive`, `fg0`.`demand_neg_rrp` `demand_neg_rrp`, `fg0`.`rrp_negative` `rrp_negative`, `fg0`.`demand_pos_rrp` `demand_pos_rrp`, `fg0`.`demand_7_mean` `demand_7_mean`, `fg0`.`demand_7_std` `demand_7_std`, `fg0`.`demand_14_mean` `demand_14_mean`, `fg0`.`demand_14_std` `demand_14_std`, `fg0`.`demand_30_mean` `demand_30_mean`, `fg0`.`demand_30_std` `demand_30_std`
FROM `electricity_demand_featurestore`.`electricity_fg_1` `fg0`
WHERE `fg0`.`demand` > 10000


,index,date,rrp,frac_at_neg_rrp,demand,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,1886,1583020800000,43.745921,0.000000,101413.145,43.745921,0.000,0.000000,101413.145,110717.865714,9220.740759,110795.223214,8592.274480,113918.470167,11661.928533
1,2039,1596240000000,47.915895,0.000000,111590.650,47.915895,0.000,0.000000,111590.650,127876.293571,7974.452809,130397.305714,8870.056804,131883.940500,8381.503670
2,1347,1536451200000,67.615591,0.104167,107838.950,77.332217,9927.775,-28.213155,97911.175,119398.098571,8876.580027,124830.694643,10492.613383,125011.080833,9375.466267
3,634,1474848000000,47.671470,0.000000,124343.190,47.671470,0.000,0.000000,124343.190,120814.523571,10247.936071,122961.179286,10768.576673,123261.361000,9562.472546
4,1435,1544054400000,125.308108,0.000000,135452.260,125.308108,0.000,0.000000,135452.260,109817.412143,14171.194821,108530.186071,10633.545758,107938.950000,9656.161129


---

# <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [10]:
[t_func.name for t_func in fs.get_transformation_functions()]

['min_max_scaler', 'robust_scaler', 'label_encoder', 'standard_scaler']

We can retrieve transformation function we need .

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [11]:
# Load transformation functions.
standard_scaler = fs.get_transformation_function(name = 'standard_scaler')
label_encoder = fs.get_transformation_function(name = 'label_encoder')

#Map features to transformations.
mapping_transformers = {
    "rrp_positive": standard_scaler,
    "rrp_negative": standard_scaler,
    "school_day": label_encoder,
    "holiday": label_encoder
}

---

## <span style="color:#ff5f27;">💼 Query Preparation</span>

In [12]:
fg_query = fg_weather.select_all()\
                        .join(
                            fg_calendar.select_all(),
                            on = ['index','date']
                        )\
                        .join(
                            fg_electricity.select_all(),
                            on = ['index','date']
                        )
fg_query.show(5)

2022-07-26 08:46:15,248 INFO: USE `electricity_demand_featurestore`
2022-07-26 08:46:15,947 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg2`.`index` `index`, `fg2`.`date` `date`, `fg2`.`min_temperature` `min_temperature`, `fg2`.`max_temperature` `max_temperature`, `fg2`.`solar_exposure` `solar_exposure`, `fg2`.`rainfall` `rainfall`, `fg2`.`day_of_week` `day_of_week`, `fg2`.`day_of_month` `day_of_month`, `fg2`.`day_of_year` `day_of_year`, `fg2`.`week_of_year` `week_of_year`, `fg2`.`month` `month`, `fg2`.`quarter` `quarter`, `fg2`.`year` `year`, `fg2`.`index` `join_pk_index`, `fg2`.`date` `join_evt_date`, `fg0`.`school_day` `school_day`, `fg0`.`holiday` `holiday`, RANK() OVER (PARTITION BY `fg2`.`index`, `fg2`.`date`, `fg2`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `electricity_demand_featurestore`.`weather_fg_1` `fg2`
INNER JOIN `electricity_demand_featurestore`.`calendar_fg_1` `fg0` ON `fg2`.`index` = `fg0`.`index` AND `fg2`.`date` = `fg0`.`date` AND `fg2`.`date` >

,index,date,min_temperature,max_temperature,solar_exposure,rainfall,day_of_week,day_of_month,day_of_year,week_of_year,...,rrp_positive,demand_neg_rrp,rrp_negative,demand_pos_rrp,demand_7_mean,demand_7_std,demand_14_mean,demand_14_std,demand_30_mean,demand_30_std
0,33,1422921600000,16.1,20.0,22.7,0.0,1,3,34,6,...,27.003832,0.00,0.00,122880.130,114667.386429,8920.937447,119938.167857,16864.969397,120125.450167,15880.221937
1,34,1423008000000,14.8,19.0,21.2,0.0,2,4,35,6,...,27.809134,0.00,0.00,117398.030,114758.001429,8948.957583,117702.133929,14693.328802,120100.977833,15883.964829
2,43,1423785600000,16.1,32.4,14.9,0.0,4,13,44,7,...,26.322857,2992.08,-318.66,133078.540,133396.924286,12386.475233,125776.359286,14840.586153,122416.020000,15771.398868
3,47,1424131200000,17.6,20.6,8.3,0.0,1,17,48,8,...,30.250149,0.00,0.00,127666.010,131606.946429,9410.194936,129983.664643,10731.392554,124825.231000,14362.078276
4,50,1424390400000,17.4,25.3,24.7,0.0,4,20,51,8,...,33.538672,0.00,0.00,139465.675,129902.357857,7431.589408,131649.641071,9979.447032,125168.042667,14088.383256


---

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [13]:
feature_view = fs.create_feature_view(
    name = 'electricity_feature_view',
    version = 1,
    labels = ['demand'],
    query = fg_query
)

Feature view created successfully, explore it at 
https://0f060790-06a4-11ed-8aed-d1422d4ec537.cloud.hopsworks.ai/p/2168/fs/2116/fv/electricity_feature_view/version/1


In [14]:
feature_view

For now `Feature View` is saved in Hopsworks and we can retrieve it using `FeatureStore.get_feature_view()`.

In [15]:
feature_view = fs.get_feature_view(
    name = 'electricity_feature_view',
    version = 1
)

In [16]:
feature_view.version

1

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset we use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- We can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset</span>

In [17]:
feature_view.create_training_data(
    description = 'training_dataset',
    data_format = 'csv'
)

Training dataset job started successfully, you can follow the progress at 
https://0f060790-06a4-11ed-8aed-d1422d4ec537.cloud.hopsworks.ai/p/2168/jobs/named/electricity_feature_view_1_1_create_fv_td_26072022084643/executions


(1, <hsfs.core.job.Job at 0x7f75bda29d90>)

- We can create **train, test** splits using `create_train_test_split()`. 

- We can create **train,validation, test** splits using `create_train_validation_test_splits()` methods.

- The only thing is that we should specify desired ratio of splits.

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train and test splits</span>

In [18]:
feature_view.create_train_test_split(
    test_size = 0.2
)

Training dataset job started successfully, you can follow the progress at 
https://0f060790-06a4-11ed-8aed-d1422d4ec537.cloud.hopsworks.ai/p/2168/jobs/named/electricity_feature_view_1_2_create_fv_td_26072022084933/executions


(2, <hsfs.core.job.Job at 0x7f75bdb70fd0>)

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train, validation and test splits</span>

In [ ]:
feature_view.create_train_validation_test_split(
    validation_size = 0.2,
    test_size = 0.1
)

---

## <span style="color:#ff5f27;"> 🪝 Retrieving Datasets </span>

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset</span>

In [ ]:
X_train, y_train = feature_view.get_training_data(
    training_dataset_version = 1
)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_train.shape

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train and test splits</span>

In [ ]:
X_train, y_train, X_test, y_test = feature_view.get_train_test_split(
    training_dataset_version = 2
)

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### <span style="color:#ff5f27;"> ⛳️ Dataset with train, validation and test splits</span>

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = feature_view.get_train_validation_test_split(
    training_dataset_version = 3
)

In [ ]:
X_train.head()

---

## <span style="color:#ff5f27;"> 🔮 Creating Training Datasets with Event Time filter</span>

First of all lets import **datetime** from datetime library and set up a time format.

Then we can define start_time point and end_time point.

Finally we can create training dataset with data in specific time bourders. 


In [ ]:
from datetime import datetime

def from_unix_to_datetime(unix):
    return datetime.utcfromtimestamp(unix).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
date_format = '%Y-%m-%d %H:%M:%S'

start_time_train = int(float(datetime.strptime('2017-01-01 00:00:01',date_format).timestamp()) * 1000)
end_time_train = int(float(datetime.strptime('2018-02-01 23:59:59',date_format).timestamp()) * 1000)

start_time_test = int(float(datetime.strptime('2018-02-02 23:59:59',date_format).timestamp()) * 1000)
end_time_test = int(float(datetime.strptime('2019-02-01 23:59:59',date_format).timestamp()) * 1000)

#### <span style="color:#ff5f27;"> ⛳️ Simple Training Dataset with event time</span>

In [ ]:
feature_view.create_training_data(
    description = 'data_2017_2018',
    data_format = 'csv',
    start_time = start_time_train,
    end_time = end_time_train
)

In [ ]:
X_train_lim, y_train_lim = feature_view.get_training_data(
    training_dataset_version = 4
)

In [ ]:
(X_train_lim.date.agg({'min','max'}) / 1000).apply(lambda x: from_unix_to_datetime(x))

#### <span style="color:#ff5f27;"> ⛳️ Training Dataset with train and test splits with event time</span>

In [ ]:
# feature_view.create_train_test_split(
#     test_size = 0.2,
#     train_start = start_time_train,
#     train_end = end_time_train,
#     test_start = start_time_test,
#     test_end = end_time_test
# )

---

### <span style="color:#ff5f27;"> Next Steps</span>

In the next notebook, we will train a model on the Training Dataset we created in this notebook.